In [2]:
import numpy as np
import pandas as pd
import json
from pandas.io.json import json_normalize
import boto3
import json
import seaborn as sns
from matplotlib import pyplot as plt
from collections import Counter
import networkx as nx
import re
from node2vec import Node2Vec

In [91]:
# from sagemaker import get_execution_role
# role = get_execution_role()

## Load Raw Data CSV from S3 into Pandas DataFrame

In [92]:
#bucket = 'lndworkml'
#conn = boto3.client('s3')
#s3_clientobj = conn.get_object(Bucket=bucket, Key='knowledgegraphml/Bruce_4.csv')

In [4]:
df = pd.concat(pd.read_excel('timestamp-data-5-25.xlsx', sheet_name=None), ignore_index=True)

In [5]:
df

,Podcast Title,Season,Episode #,Episode Title,Date of Episode,URL,Guest Name(s),Timestamp,Clip Description,Topic
0,Lightning Junkies,NaN,44.0,Exploring RBC w/ Giacomo Zucco,"May 14th, 2021",https://anchor.fm/lightning-junkies/episodes/E...,Giacomo Zucco,00:00:00,Intro,NaN
1,Lightning Junkies,NaN,44.0,Exploring RBC w/ Giacomo Zucco,"May 14th, 2021",https://anchor.fm/lightning-junkies/episodes/E...,Giacomo Zucco,00:03:27,What is [[RGB]]?,NaN
2,Lightning Junkies,NaN,44.0,Exploring RBC w/ Giacomo Zucco,"May 14th, 2021",https://anchor.fm/lightning-junkies/episodes/E...,Giacomo Zucco,00:08:40,Why [[RGB]]?,NaN
3,Lightning Junkies,NaN,44.0,Exploring RBC w/ Giacomo Zucco,"May 14th, 2021",https://anchor.fm/lightning-junkies/episodes/E...,Giacomo Zucco,00:15:23,Would [[RGB]] work better with [[Eltoo]]/[[SIG...,NaN
4,Lightning Junkies,NaN,44.0,Exploring RBC w/ Giacomo Zucco,"May 14th, 2021",https://anchor.fm/lightning-junkies/episodes/E...,Giacomo Zucco,00:17:19,[[Transaction Fees]] on Bitcoin/ Could [[RGB]]...,NaN
...,...,...,...,...,...,...,...,...,...,...
20152,Bitcoin Explained - The Technical Side of Bitcoin,NaN,1.0,Explaining New PSBT and RBF Attacks,"Sep 22nd, 2020",https://podcasts.apple.com/nl/podcast/explaini...,NaN,00:26:22,NaN,Ongoing debate to make it deliberately unsafe ...
20153,Bitcoin Explained - The Technical Side of Bitcoin,NaN,1.0,Explaining New PSBT and RBF Attacks,"Sep 22nd, 2020",https://podcasts.apple.com/nl/podcast/explaini...,NaN,00:31:23,NaN,[[Bitcoin Core]]'s workaround to not allow spe...
20154,Bitcoin Explained - The Technical Side of Bitcoin,NaN,1.0,Explaining New PSBT and RBF Attacks,"Sep 22nd, 2020",https://podcasts.apple.com/nl/podcast/explaini...,NaN,00:33:26,NaN,Using security vulnerabilities as a marketing ...
20155,Bitcoin Explained - The Technical Side of Bitcoin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df.columns

Index(['Podcast Title', 'Season', 'Episode #', 'Episode Title',
       'Date of Episode', 'URL', 'Guest Name(s)', 'Timestamp',
       'Clip Description', 'Topic'],
      dtype='object')

## Parse Out Topic Nodes and Indexed Content Nodes

In [7]:
indexed_content_nodes = []
topic_nodes = []
edge_tuples = []

for index, row in df.iterrows():
    if  index == df.index[-1]:
        indexed_content_title = f"{row['Timestamp']}-end, {row['URL']} "
    else:
        indexed_content_title = f"{row['Timestamp']}-{df.iloc[index+1]['Timestamp']}, {row['URL']} "
    indexed_content_nodes.append(indexed_content_title)
    res = re.findall('(?<=\[\[)([^\[\]]*)(?=\]\])', str(row['Clip Description']))
    topic_nodes_lowercase = list((map(lambda x: x.lower(), res)))
    topic_nodes.extend(topic_nodes_lowercase)
    for top in topic_nodes_lowercase:
        edge_tuples.append((indexed_content_title, top))
        

In [8]:
B = nx.Graph()
B.add_nodes_from(list(set(indexed_content_nodes)), bipartite=0)
B.add_nodes_from(list(set(topic_nodes)), bipartite=1)

In [9]:
indexed_content_nodes[20:40]

['00:03:26-00:03:53, https://www.youtube.com/watch?v=-yiTtO_p3Cw ',
 '00:03:53-00:05:41, https://www.youtube.com/watch?v=-yiTtO_p3Cw ',
 '00:05:41-00:06:19, https://www.youtube.com/watch?v=-yiTtO_p3Cw ',
 '00:06:19-00:00:00, https://www.youtube.com/watch?v=-yiTtO_p3Cw ',
 '00:00:00-00:04:13, https://podcasts.apple.com/us/podcast/stablecoin-lunacy-with-brad-mills/id1261133600?i=1000561495707 ',
 '00:04:13-00:05:46, https://podcasts.apple.com/us/podcast/stablecoin-lunacy-with-brad-mills/id1261133600?i=1000561495707 ',
 '00:05:46-00:06:37, https://podcasts.apple.com/us/podcast/stablecoin-lunacy-with-brad-mills/id1261133600?i=1000561495707 ',
 '00:06:37-00:12:47, https://podcasts.apple.com/us/podcast/stablecoin-lunacy-with-brad-mills/id1261133600?i=1000561495707 ',
 '00:12:47-00:13:46, https://podcasts.apple.com/us/podcast/stablecoin-lunacy-with-brad-mills/id1261133600?i=1000561495707 ',
 '00:13:46-00:17:53, https://podcasts.apple.com/us/podcast/stablecoin-lunacy-with-brad-mills/id12611336

In [10]:
B.add_edges_from(edge_tuples)

In [10]:
B

In [11]:
nx.info(B)

C:\Users\zhang\AppData\Local\Temp\ipykernel_21924\770005836.py:1: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  nx.info(B)


'Graph with 27774 nodes and 20641 edges'

## Neighbor Search for a Topic

In [12]:
for e in B.edges('ghj'):
    print(e[1])

## Embed Graph Using Node2Vec and Get Similarity Score

In [13]:
node2vec = Node2Vec(B, dimensions=64, walk_length=5, num_walks=200, workers=4)  

Computing transition probabilities:   0%|          | 0/27774 [00:00<?, ?it/s]

In [14]:
# WAIT!!!!!

model = node2vec.fit(window=10, min_count=1, batch_words=4)

In [15]:
model.save("word2vec.model")

In [234]:
# for node in model.wv.most_similar('atomic swap'):
#     print(node[0])

00:20:44-00:21:25, https://podcasts.apple.com/us/podcast/is-algorand-the-future-of-finance/id1261133600?i=1000552767016 
00:41:03-00:48:36, https://noded.org/podcast/noded-0450-bitstein-on-the-bitcoin-echo-chamber-podcast-with-heavily-armed-clown/ 
lightcoin
00:10:00-00:13:29, https://podcasts.apple.com/us/podcast/trey-griffith-from-sparkswap-lnj04/id1477726305?i=1000451757320 
00:56:07-00:58:58, https://podcasts.apple.com/us/podcast/guys-take-49-backed-by-truth/id1359544516?i=1000531279477 
00:33:39-00:44:50, nan 
00:12:12-00:13:14, https://podcasts.apple.com/us/podcast/read-540-bitcoin-the-american-idea-alex-gladstein/id1359544516?i=1000527920941 
01:03:30-01:05:15, https://podcasts.apple.com/us/podcast/read-540-bitcoin-the-american-idea-alex-gladstein/id1359544516?i=1000527920941 
00:25:21-00:27:54, https://onthebrink-podcast.com/iris/ 
00:28:29-00:34:22, https://www.whatbitcoindid.com/podcast/the-breakdown-of-trust 
